# RCy3を用いたCytoscapeの自動化

著者:
- 西田 孝三 [理研],
- Alex Pico [Gladstone Institute].

最終変更: 2021年10月22日

## 概要

### 説明

このワークショップでは、ネットワーク可視化ソフトCytoscapeの操作を自動化する方法を教えます。
操作の自動化はRCy3というBioconductorパッケージを用いて行います。
このワークショップを学ぶことで学生は...ができるようになります。
ワークショップの種類はインストラクター主導のライブデモです。

### ワークショップの前提条件

* R構文の基本的な知識
* Cytoscape Desktop のローカル環境へのインストール
* ネットワーク可視化の基本的な知識
* ほげ

#### ワークショップに関連する背景資料

* Otasek, David, et al. “Cytoscape Automation: Empowering Workflow-Based Network Analysis.” Genome Biology, vol. 20, no. 1, Dec. 2019, p. 185. https://doi.org/10.1186/s13059-019-1758-4.
* Gustavsen, Julia A., et al. “RCy3: Network Biology Using Cytoscape from within R.” F1000Research, vol. 8, Nov. 2019, p. 1774. https://doi.org/10.12688/f1000research.20887.2.

### どのように参加するか

1. [Cytoscape Desktop](https://cytoscape.org/) をインストール
2. [Orchestra ワークショッププラットフォーム](http://app.orchestra.cancerdatasci.org/1)をGoogle Chromeで開く
3. Orchestra で立ち上げられる Jupyter Notebook で workshop.ipynb 中のセルを実行する

### 用いる _R_ / _Bioconductor_ パッケージ

- RCy3

### 時間の概要

| Activity                     | Time |
|------------------------------|------|
| Cytoscape Automation         | 15m  |
| Automation Use Cases         | 20m  |
| Wrap-up and "Exam"           | 5m   |

### ワークショップの目標と目的

このワークショップが終了するまでに、次のことができるようになります:

- Cytoscapeに対するコマンドプログラムによる制御
- Cytoscapeをバイオインフォマティクスパイプラインに統合する
- ネットワーク可視化ワークフローをオンラインで公開、共有する

### 学習目標

* describe how to...
* identify methods for...
* understand the difference between...
* analyze xyz data to produce...
* create xyz plots
* evaluate xyz data for artifacts

## ワークショップ

Divide the workshop into sections (`## A Section`). Include
fully-evaluated _R_ code chunks. Develop exercises and solutions, and
anticipate that your audience will walk through the code with you, or
work on the code idependently -- do not be too ambitious in the
material that you present.